## Generate Data

#### Store Data
Generate store front data

In [1]:
import folium
import numpy as np
import pandas as pd

In [164]:
class BaseEntity:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

    def __repr__(self):
        attrs = ', '.join(f"{key}={value}" for key, value in self.__dict__.items())
        return f"{self.__class__.__name__}({attrs})"
    
    def showDataFrame(self):
        return pd.DataFrame([self.__dict__])


class Store(BaseEntity):
    all_stores = []  # Class-level list to store all instances of Store

    def __init__(self, store_id, latitude, longitude, store_name, opendt):
        super().__init__(STORE_ID=store_id, 
                         LATITUDE=latitude, 
                         LONGITUDE=longitude, 
                         STORE_NAME=store_name, 
                         OPENDT=opendt)
        Store.all_stores.append(self)  # Add each instance to the class-level list

    def __repr__(self):
        return f"Store(store_id={self.STORE_ID}, lat={self.LATITUDE}, lon={self.LONGITUDE}, name={self.STORE_NAME}, opendt={self.OPENDT})"

    @classmethod
    def display_all_stores(cls):
        """
        Display a DataFrame of all store instances.
        """
        return pd.DataFrame([store.__dict__ for store in cls.all_stores])
    

class Customer(BaseEntity):
    all_customers = []  # Class-level list to store all instances of Customers
    def __init__(self, customer_id, latitude, longitude, firstname, lastname, homestore_ID):
        super().__init__(CUSTOMER_ID=customer_id, 
                         LATITUDE=latitude, 
                         LONGITUDE=longitude, 
                         FIRSTNAME=firstname, 
                         LASTNAME=lastname, 
                         HOMESTORE_ID=homestore_ID)
        Customer.all_customers.append(self)  # Add each instance to the class-level list

    def __repr__(self):
        return f"Customer(customer_id={self.CUSTOMER_ID}, lat={self.LATITUDE}, lon={self.LONGITUDE}, name={self.FIRSTNAME} {self.LASTNAME}, homestoreID={self.HOMESTORE_ID})"
    
    @classmethod
    def display_all_customers(cls):
        """
        Display a DataFrame of all customer instances.
        """
        return pd.DataFrame([cust.__dict__ for cust in cls.all_customers])
    
class Order(BaseEntity):
    all_orders = []  # Class-level list to store all instances of Orders
    def __init__(self, order_id, customer_id, store_id, order_date, subtotal,tax, total):        
        super().__init__(ORDER_ID=order_id, 
                         CUSTOMER_ID=customer_id, 
                         STORE_ID=store_id, 
                         ORDER_DATE=order_date, 
                         SUBTOTAL=subtotal,
                         TAX=tax,
                         TOTAL=total)

    @classmethod
    def display_all_orders(cls):
        """
        Display a DataFrame of all order instances.
        """
        return pd.DataFrame([order.__dict__ for order in cls.all_orders])

#### Generate random store locations

In [165]:
min_lat, max_lat = 33.0633, 33.2182
min_lon, max_lon = -96.9162, -96.6718

num_stores = 10

import random
random.seed(1033)  # For reproducibility

In [166]:
# Generate random store locations within the bounding box


Store.all_stores = []  # Reset the class-level list to avoid duplicates
name_prefix = "Panucci's Pizza - "
store_ids = [random.randint(0, 9999) for _ in range(num_stores)]
store_locations = [
    Store(
        store_id=store_id,
        latitude=np.round(random.uniform(min_lat, max_lat),5),
        longitude=np.round(random.uniform(min_lon, max_lon),5),
        store_name=f"{name_prefix}{store_id}",
        opendt=f"{random.randint(2010, 2023)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}"
    ).__dict__ for store_id in store_ids
]

Store.display_all_stores()

,STORE_ID,LATITUDE,LONGITUDE,STORE_NAME,OPENDT
0,3431,33.18667,-96.87610,Panucci's Pizza - 3431,2023-03-22
1,4464,33.12457,-96.87434,Panucci's Pizza - 4464,2016-02-21
2,7496,33.14074,-96.80274,Panucci's Pizza - 7496,2010-07-21
3,1119,33.21054,-96.84067,Panucci's Pizza - 1119,2015-11-28
4,1249,33.10539,-96.87740,Panucci's Pizza - 1249,2021-05-01
5,3789,33.07664,-96.90281,Panucci's Pizza - 3789,2010-12-01
6,151,33.14574,-96.74385,Panucci's Pizza - 151,2016-06-20
7,6202,33.09622,-96.70679,Panucci's Pizza - 6202,2015-01-18
8,6144,33.16461,-96.82926,Panucci's Pizza - 6144,2019-11-28
9,2510,33.10672,-96.84545,Panucci's Pizza - 2510,2022-03-06


In [167]:
# # Create a map centered around Frisco, TX
# map_center = [np.mean([min_lat,max_lat]), np.mean([min_lon,max_lon])]
# m = folium.Map(location=map_center, zoom_start=12, tiles='cartodbdark_matter')

# # Add markers for each store location
# for location in Store.all_stores:
#     # Create a marker for each store location
#     # Add a popup with the store name and open date
#     folium.Marker(
#         [location.LATITUDE, location.LONGITUDE],
#         popup=folium.Popup(f"{location.STORE_NAME}", max_width=1000),
#         tooltip=location.STORE_NAME,
#     ).add_to(m)

# m

#### Generate customer locations for each store

In [168]:
num_customers_per_store = (20,50)

radius_range = (0.5, 2.0)  # in km

In [169]:
from scipy.stats import gamma
import numpy as np

# Predefined list of first names
first_names = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Hannah", "Ivy", "Jack", "Kathy", "Liam", "Mona", "Nathan", "Olivia", "Paul", "Quincy", "Rachel", "Steve", "Tina"]
# Predefined list of Pokémon names
pokemon_last_names = ["Pikachu", "Charmander", "Bulbasaur", "Squirtle", "Jigglypuff", "Meowth", "Psyduck", "Snorlax", "Eevee", "Mewtwo"]


def generate_customers(store_location, search_radius, num_customers, store_id=None):
    """
    Generate customers around a store location using a gamma-gamma distribution.

    :param store_location: Tuple of (latitude, longitude) for the store location.
    :param search_radius: Maximum search radius in kilometers.
    :param num_customers: Number of customers to generate.
    :param alpha: Shape parameter for the gamma distribution (default: 2.0).
    :param beta: Scale parameter for the gamma distribution (default: 2.0).
    :param store_id: ID of the store to associate customers with.
    :return: List of Customer objects.
    """
    customers = []
    for i in range(num_customers):
        # Generate a random distance using the exponential distribution
        distance = np.random.exponential(scale=search_radius / 2)
        # Generate a random bearing (angle in radians)
        bearing = np.random.uniform(0, 2 * np.pi)
        # Calculate the new latitude and longitude
        delta_lat = distance * np.cos(bearing) / 111  # Approx. conversion of km to degrees latitude
        delta_lon = distance * np.sin(bearing) / (111 * np.cos(np.radians(store_location[0])))  # Adjust for longitude
        customer_lat = store_location[0] + delta_lat
        customer_lon = store_location[1] + delta_lon
        # Create a Customer object
        customer_id = f"{random.randint(0, 9999)}"
        first_name = random.choice(first_names)
        last_name = random.choice(pokemon_last_names)
        customers.append(Customer(customer_id, np.round(customer_lat,5), np.round(customer_lon,5), first_name, last_name, store_id))
    return customers

In [170]:
# Generate customers for each store
Customer.all_customers = []  # Reset the class-level list to avoid duplicates


for store in Store.all_stores:
    store_location = (store.LATITUDE, store.LONGITUDE)
    num_customers = random.randint(*num_customers_per_store)  # Randomly choose the number of customers within the range
    search_radius_km = random.uniform(*radius_range)  # Randomly choose the search radius within the range

    generate_customers(store_location, search_radius_km, num_customers, store_id=store.STORE_ID)


In [171]:
Customer.display_all_customers()

,CUSTOMER_ID,LATITUDE,LONGITUDE,FIRSTNAME,LASTNAME,HOMESTORE_ID
0,7336,33.18770,-96.87076,Bob,Mewtwo,3431
1,928,33.18663,-96.87602,Ivy,Jigglypuff,3431
2,2835,33.18637,-96.87625,Rachel,Meowth,3431
3,4351,33.18712,-96.86948,Frank,Meowth,3431
4,9372,33.17848,-96.88722,Nathan,Eevee,3431
...,...,...,...,...,...,...
298,7675,33.10669,-96.84651,Hannah,Pikachu,2510
299,6365,33.10688,-96.84619,Mona,Charmander,2510
300,1589,33.10660,-96.84557,Olivia,Meowth,2510
301,6512,33.10865,-96.85091,Liam,Charmander,2510


#### Transactional data
Generate random transactional data for each store

In [172]:
def generate_gaussian_value(mean, std, floor):
    """
    Generate a random value based on a Gaussian distribution.

    :param mean: Mean of the Gaussian distribution.
    :param std: Standard deviation of the Gaussian distribution.
    :param floor: Minimum value (floor) for the generated value.
    :return: A random value from the Gaussian distribution, floored at the specified minimum value.
    """
    value = np.random.normal(loc=mean, scale=std)
    return round(max(value, floor), 2)  # Round to 2 decimal places and ensure it's not below the floor value

In [181]:
num_orders_per_customer = (3, 25)

Order.all_orders = []  # Reset the class-level list to avoid duplicates

for customer in Customer.all_customers:
    num_orders = random.randint(*num_orders_per_customer)  # Randomly choose the number of orders for each customer
    for _ in range(num_orders):
        order_id = f"{random.randint(0, 99999)}"
        order_date = f"{random.randint(2020, 2023)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}"
        order_subtotal = generate_gaussian_value(mean=30, std=10, floor=5)  # Generate a random order total
        order_tax = np.round(order_subtotal * 0.0825,2)  # Assuming a tax rate of 8.25%
        order_total = np.round(order_subtotal + order_tax,2)
        Order.all_orders.append(Order(order_id, customer.CUSTOMER_ID, customer.HOMESTORE_ID, order_date, order_subtotal, order_tax, order_total))

In [182]:
Order.display_all_orders()

,ORDER_ID,CUSTOMER_ID,STORE_ID,ORDER_DATE,SUBTOTAL,TAX,TOTAL
0,89743,7336,3431,2022-01-27,20.84,1.72,22.56
1,5308,7336,3431,2020-12-03,37.91,3.13,41.04
2,19320,7336,3431,2023-07-13,21.93,1.81,23.74
3,59079,7336,3431,2023-08-21,26.57,2.19,28.76
4,22260,7336,3431,2021-05-19,22.04,1.82,23.86
...,...,...,...,...,...,...,...
4180,5966,4539,2510,2023-04-03,37.50,3.09,40.59
4181,17299,4539,2510,2021-04-12,28.03,2.31,30.34
4182,79240,4539,2510,2021-03-18,31.69,2.61,34.30
4183,49721,4539,2510,2021-06-27,12.10,1.00,13.10


### Create SnowFlake Data

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
SF_USER = os.getenv("SF_USER")
SF_PASSWORD = os.getenv("SF_PASSWORD")
SF_ACCOUNT = os.getenv("SF_ACCOUNT")

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as SF

connection_parameters = {
  "account": SF_ACCOUNT,
  "user": SF_USER,
  "password": SF_PASSWORD,
}
new_session = Session.builder.configs(connection_parameters).create()

In [143]:
new_db = "PANUCCIS_PIZZA"
new_schema = "POS_MAIN"
new_WH = "PANUCCIS_PIZZA_WH"
table_stores = "STORES"
table_customers = "CUSTOMERS"
table_orders = "ORDERS_HEADER"

In [144]:
# create warehouse
new_session.sql(f"""
                CREATE WAREHOUSE if not exists {new_WH}
                WITH
                WAREHOUSE_SIZE = XSMALL
                AUTO_SUSPEND = 30
                INITIALLY_SUSPENDED = TRUE
                COMMENT = 'WH TO DO TEST RUNS ON PANUCCIS PIZZA DB'
                """).collect()

new_session.use_warehouse(f"{new_WH}")

# create the Panuccis Database
new_session.sql(f'CREATE OR REPLACE DATABASE {new_db};').collect()

# create the Panuccis PoS Main Schema
new_session.sql(f'CREATE OR REPLACE SCHEMA {new_db}.{new_schema};').collect()

[Row(status='Schema POS_MAIN successfully created.')]

In [184]:
# Convert customer data to a pandas DataFrame
customer_df = Customer.display_all_customers()
orders_df = Order.display_all_orders().sort_values(by='ORDER_DATE', ascending=False)
store_df = Store.display_all_stores()

# Add the first order date for each customer
first_order_dates = orders_df.groupby(['CUSTOMER_ID'])[['ORDER_DATE']].min().rename(columns={'ORDER_DATE':'FIRST_ORDER_DATE'})
customer_df = customer_df.join(first_order_dates, on='CUSTOMER_ID', how='left')


customer_df = customer_df.sort_values(by='FIRST_ORDER_DATE', ascending=False)

In [186]:
from snowflake.snowpark.types import StructType, StructField, StringType, FloatType, IntegerType, DateType

# Create a Snowpark DataFrame from the pandas DataFrame
schema = StructType([
    StructField("CUSTOMER_ID", StringType()),
    StructField("LATITUDE", FloatType()),
    StructField("LONGITUDE", FloatType()),
    StructField("FIRSTNAME", StringType()),
    StructField("LASTNAME", StringType()),
    StructField("HOMESTORE_ID", IntegerType()),
    StructField("FIRST_ORDER_DATE", DateType())
])
# Write the data to the Snowflake table
snowpark_customer_df = new_session.create_dataframe(customer_df.to_records(index=False).tolist(), 
                                                    schema=schema)
snowpark_customer_df.write.mode("overwrite").save_as_table(f"{new_db}.{new_schema}.{table_customers}")




# Define the schema for the store data
schema = StructType([
    StructField("STORE_ID", IntegerType()),
    StructField("LATITUDE", FloatType()),
    StructField("LONGITUDE", FloatType()),
    StructField("STORE_NAME", StringType()),
    StructField("OPENDT", DateType())
])
# Write the data to the Snowflake table
snowpark_store_df = new_session.create_dataframe(store_df.to_records(index=False).tolist(), 
                                                 schema=schema)
snowpark_store_df.write.mode("overwrite").save_as_table(f"{new_db}.{new_schema}.{table_stores}")




# Define the schema for the orders data
schema = StructType([
    StructField("ORDER_ID", StringType()),
    StructField("CUSTOMER_ID", StringType()),
    StructField("STORE_ID", IntegerType()),
    StructField("ORDER_DATE", DateType()),
    StructField("SUBTOTAL", FloatType()),
    StructField("TAX", FloatType()),
    StructField("TOTAL", FloatType()),
])

# Write the data to the Snowflake table
snowpark_orders_df = new_session.create_dataframe(orders_df.to_records(index=False).tolist(), 
                                                  schema=schema)
snowpark_orders_df.write.mode("overwrite").save_as_table(f"{new_db}.{new_schema}.{table_orders}")

#### Snowflake CleanUp
When done with the exercise use the following code to clean up your snowflake enviornment

In [188]:
new_session.sql(f"DROP TABLE IF EXISTS {new_db}.{new_schema}.{table_stores};").collect()
new_session.sql(f"DROP TABLE IF EXISTS {new_db}.{new_schema}.{table_customers};").collect()
new_session.sql(f"DROP TABLE IF EXISTS {new_db}.{new_schema}.{table_orders};").collect()
new_session.sql(f"DROP SCHEMA IF EXISTS {new_db}.{new_schema};").collect()
new_session.sql(f"DROP DATABASE IF EXISTS {new_db};").collect()
new_session.sql(f"DROP WAREHOUSE IF EXISTS {new_WH};").collect()
new_session.close()